In [ ]:
pip install pandas pyarrow

In [1]:
import json
import pandas as pd
import gzip
import ast
from datetime import datetime
import numpy as np


In [2]:
# Desarrollo función leer_Json_gzip() que lee los archivos 'comprimidos.json.gz` y devuelve una lista de diccionarios Python.
def leer_json_gzip(users_items):

    # Abrir el archivo users_items.json.gz en modo de lectura y decodificamos el contenido del archivo.
    with gzip.open(users_items, 'rt', encoding='utf-8') as miArchivo:

        # Recorremos el contenido del archivo y evaluamos cada línea como una expresión Python.
        # El resultado de la evaluación es un diccionario Python.
        # La función strip() elimina los espacios en blanco al principio y al final de la línea.
        return [ast.literal_eval(line.strip()) for line in miArchivo]
    #Usamos ast.literal_eval para convertir cada linea en un diccionario y lo agregamos a la lista

In [3]:
datos_users_items = []
for i  in gzip.open('users_items.json.gz'):
    datos_users_items.append(ast.literal_eval(i.decode('utf-8')))

df_users_items = pd.DataFrame(datos_users_items)


In [ ]:
df_users_items

In [9]:
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [7]:
df_users_items.size

441550

In [6]:
df_users_items = df_users_items.dropna()

In [ ]:
df_users_items

In [8]:
# Al hacer la función veo que debo desanidar columna "items" para encontrar el tiempo jugado

df_users_items_exploded = df_users_items.explode('items')

In [ ]:
df_users_items_exploded

In [10]:
# Normalizo los datos y creo un nuevo data frame desanidado
df_users_items_desanidado = pd.json_normalize(df_users_items_exploded['items'].dropna())

In [11]:
# Volver a indexar
df_users_items_exploded.reset_index(inplace=True)


In [12]:
#  Concateno Data Frames y borro columna original items
df_users_items_concatenado = pd.concat([df_users_items_exploded, df_users_items_desanidado], axis=1)


In [ ]:
df_users_items_concatenado

In [14]:
df_users_items_concatenado =  df_users_items_concatenado.drop(columns=['items'])

In [ ]:
df_users_items_concatenado

In [ ]:
df_users_items_concatenado.info()

In [17]:
# Renombro nuevamente
df_users_items = df_users_items_concatenado

In [ ]:
df_users_items

In [19]:
# La columnas index
df_users_items =df_users_items.drop("steam_id", axis= 1)

In [20]:
# Borar columna index que tienen el mismo titulo y diferente info

df_users_items = df_users_items.drop("index",axis=1)

In [ ]:
df_users_items

In [ ]:
df_users_items.info()

In [23]:
df_users_item = df_users_items.dropna()

In [24]:
df_users_items = df_users_items.drop_duplicates()

In [25]:
df_users_items = df_users_items.dropna()

In [ ]:
df_users_items 

In [27]:
# Guardo el df en parquet

df_users_items.to_parquet('df_users_items.parquet')